In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 256
tree_depth = 8
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.134346008300781 | KNN Loss: 6.232436656951904 | BCE Loss: 1.9019089937210083
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.142910957336426 | KNN Loss: 6.23236083984375 | BCE Loss: 1.9105498790740967
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.195171356201172 | KNN Loss: 6.232297420501709 | BCE Loss: 1.962874412536621
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.213438034057617 | KNN Loss: 6.232086658477783 | BCE Loss: 1.981351375579834
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.177547454833984 | KNN Loss: 6.232169151306152 | BCE Loss: 1.945378303527832
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.185166358947754 | KNN Loss: 6.23225212097168 | BCE Loss: 1.9529139995574951
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.142288208007812 | KNN Loss: 6.232115745544434 | BCE Loss: 1.910172939300537
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.10138988494873 | KNN Loss: 6.232104778289795 | BCE Loss: 1.869284868240

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 7.31865930557251 | KNN Loss: 6.1899638175964355 | BCE Loss: 1.1286954879760742
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 7.280426025390625 | KNN Loss: 6.181915283203125 | BCE Loss: 1.098510980606079
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 7.262902736663818 | KNN Loss: 6.180129051208496 | BCE Loss: 1.0827738046646118
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 7.279411315917969 | KNN Loss: 6.1774139404296875 | BCE Loss: 1.1019973754882812
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 7.265653133392334 | KNN Loss: 6.172086715698242 | BCE Loss: 1.0935664176940918
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 7.274707794189453 | KNN Loss: 6.172283172607422 | BCE Loss: 1.1024248600006104
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 7.233590126037598 | KNN Loss: 6.16810941696167 | BCE Loss: 1.0654809474945068
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 7.221923828125 | KNN Loss: 6.165285587310791 | BCE Loss: 1.

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 6.826141834259033 | KNN Loss: 5.800817489624023 | BCE Loss: 1.0253244638442993
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 6.8504414558410645 | KNN Loss: 5.806095600128174 | BCE Loss: 1.044345736503601
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 6.882237911224365 | KNN Loss: 5.80863094329834 | BCE Loss: 1.0736068487167358
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 6.842680931091309 | KNN Loss: 5.801398277282715 | BCE Loss: 1.0412826538085938
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 6.855913162231445 | KNN Loss: 5.80914831161499 | BCE Loss: 1.046764850616455
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 6.846343040466309 | KNN Loss: 5.790359020233154 | BCE Loss: 1.0559839010238647
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 6.848967552185059 | KNN Loss: 5.792078018188477 | BCE Loss: 1.056889533996582
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 6.8172688484191895 | KNN Loss: 5.775050163269043 | BCE Loss: 

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 6.697096824645996 | KNN Loss: 5.649101257324219 | BCE Loss: 1.0479955673217773
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 6.687907695770264 | KNN Loss: 5.642653942108154 | BCE Loss: 1.0452537536621094
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 6.684504985809326 | KNN Loss: 5.637027740478516 | BCE Loss: 1.047477126121521
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 6.726513862609863 | KNN Loss: 5.665369033813477 | BCE Loss: 1.0611450672149658
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 6.746542930603027 | KNN Loss: 5.689077854156494 | BCE Loss: 1.057464838027954
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 6.70589017868042 | KNN Loss: 5.639133453369141 | BCE Loss: 1.0667567253112793
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 6.704431056976318 | KNN Loss: 5.657406806945801 | BCE Loss: 1.0470243692398071
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 6.731531620025635 | KNN Loss: 5.679787635803223 | BCE Loss: 

Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 6.752091884613037 | KNN Loss: 5.704764366149902 | BCE Loss: 1.0473275184631348
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 6.636539459228516 | KNN Loss: 5.615149021148682 | BCE Loss: 1.0213905572891235
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 6.690989971160889 | KNN Loss: 5.654325485229492 | BCE Loss: 1.036664366722107
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 6.6825852394104 | KNN Loss: 5.641018867492676 | BCE Loss: 1.041566252708435
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 6.712943077087402 | KNN Loss: 5.646514415740967 | BCE Loss: 1.066428780555725
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 6.7194294929504395 | KNN Loss: 5.655757427215576 | BCE Loss: 1.0636720657348633
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 6.678214073181152 | KNN Loss: 5.614256858825684 | BCE Loss: 1.0639570951461792
Epoch 44 / 500 | iteration 15 / 30 | Total Loss: 6.651150703430176 | KNN Loss: 5.6342315673828125 | BCE Loss: 

...

Epoch 449 / 500 | iteration 5 / 30 | Total Loss: 6.639945983886719 | KNN Loss: 5.6075263023376465 | BCE Loss: 1.0324194431304932
Epoch 449 / 500 | iteration 10 / 30 | Total Loss: 6.705848693847656 | KNN Loss: 5.629512786865234 | BCE Loss: 1.0763359069824219
Epoch 449 / 500 | iteration 15 / 30 | Total Loss: 6.694762229919434 | KNN Loss: 5.648298740386963 | BCE Loss: 1.0464632511138916
Epoch 449 / 500 | iteration 20 / 30 | Total Loss: 6.657317161560059 | KNN Loss: 5.600157737731934 | BCE Loss: 1.057159423828125
Epoch 449 / 500 | iteration 25 / 30 | Total Loss: 6.647641181945801 | KNN Loss: 5.602902412414551 | BCE Loss: 1.044739007949829
Epoch 450 / 500 | iteration 0 / 30 | Total Loss: 6.652824878692627 | KNN Loss: 5.614565849304199 | BCE Loss: 1.0382590293884277
Epoch 450 / 500 | iteration 5 / 30 | Total Loss: 6.627872943878174 | KNN Loss: 5.608248710632324 | BCE Loss: 1.0196242332458496
Epoch 450 / 500 | iteration 10 / 30 | Total Loss: 6.683035373687744 | KNN Loss: 5.635503768920898 | B

Epoch 459 / 500 | iteration 25 / 30 | Total Loss: 6.704185962677002 | KNN Loss: 5.65622091293335 | BCE Loss: 1.0479649305343628
Epoch 460 / 500 | iteration 0 / 30 | Total Loss: 6.7202558517456055 | KNN Loss: 5.668644905090332 | BCE Loss: 1.051611065864563
Epoch 460 / 500 | iteration 5 / 30 | Total Loss: 6.665330410003662 | KNN Loss: 5.607845306396484 | BCE Loss: 1.0574849843978882
Epoch 460 / 500 | iteration 10 / 30 | Total Loss: 6.696154594421387 | KNN Loss: 5.6513214111328125 | BCE Loss: 1.0448329448699951
Epoch 460 / 500 | iteration 15 / 30 | Total Loss: 6.623646259307861 | KNN Loss: 5.6089348793029785 | BCE Loss: 1.0147113800048828
Epoch 460 / 500 | iteration 20 / 30 | Total Loss: 6.674218654632568 | KNN Loss: 5.618333339691162 | BCE Loss: 1.0558851957321167
Epoch 460 / 500 | iteration 25 / 30 | Total Loss: 6.702733039855957 | KNN Loss: 5.653134346008301 | BCE Loss: 1.0495986938476562
Epoch 461 / 500 | iteration 0 / 30 | Total Loss: 6.70739221572876 | KNN Loss: 5.6386895179748535 |

Epoch 470 / 500 | iteration 15 / 30 | Total Loss: 6.616541862487793 | KNN Loss: 5.59578800201416 | BCE Loss: 1.020754098892212
Epoch 470 / 500 | iteration 20 / 30 | Total Loss: 6.731933116912842 | KNN Loss: 5.661772727966309 | BCE Loss: 1.0701605081558228
Epoch 470 / 500 | iteration 25 / 30 | Total Loss: 6.6596879959106445 | KNN Loss: 5.622954845428467 | BCE Loss: 1.0367331504821777
Epoch 471 / 500 | iteration 0 / 30 | Total Loss: 6.687159538269043 | KNN Loss: 5.616240978240967 | BCE Loss: 1.070918321609497
Epoch 471 / 500 | iteration 5 / 30 | Total Loss: 6.671160697937012 | KNN Loss: 5.635196208953857 | BCE Loss: 1.0359644889831543
Epoch 471 / 500 | iteration 10 / 30 | Total Loss: 6.667853355407715 | KNN Loss: 5.627680778503418 | BCE Loss: 1.0401724576950073
Epoch 471 / 500 | iteration 15 / 30 | Total Loss: 6.660338401794434 | KNN Loss: 5.59870719909668 | BCE Loss: 1.061631441116333
Epoch 471 / 500 | iteration 20 / 30 | Total Loss: 6.759291648864746 | KNN Loss: 5.706948757171631 | BCE

Epoch 481 / 500 | iteration 5 / 30 | Total Loss: 6.665174961090088 | KNN Loss: 5.610004901885986 | BCE Loss: 1.0551700592041016
Epoch 481 / 500 | iteration 10 / 30 | Total Loss: 6.739755153656006 | KNN Loss: 5.687113285064697 | BCE Loss: 1.0526418685913086
Epoch 481 / 500 | iteration 15 / 30 | Total Loss: 6.690363883972168 | KNN Loss: 5.615386486053467 | BCE Loss: 1.0749772787094116
Epoch 481 / 500 | iteration 20 / 30 | Total Loss: 6.667476177215576 | KNN Loss: 5.633862495422363 | BCE Loss: 1.033613681793213
Epoch 481 / 500 | iteration 25 / 30 | Total Loss: 6.6948466300964355 | KNN Loss: 5.644711971282959 | BCE Loss: 1.0501346588134766
Epoch 482 / 500 | iteration 0 / 30 | Total Loss: 6.717164039611816 | KNN Loss: 5.660238265991211 | BCE Loss: 1.0569257736206055
Epoch 482 / 500 | iteration 5 / 30 | Total Loss: 6.6404266357421875 | KNN Loss: 5.60251522064209 | BCE Loss: 1.0379115343093872
Epoch 482 / 500 | iteration 10 / 30 | Total Loss: 6.659369468688965 | KNN Loss: 5.6508355140686035 |

Epoch 491 / 500 | iteration 25 / 30 | Total Loss: 6.654203414916992 | KNN Loss: 5.602957248687744 | BCE Loss: 1.0512464046478271
Epoch 492 / 500 | iteration 0 / 30 | Total Loss: 6.699536323547363 | KNN Loss: 5.646448135375977 | BCE Loss: 1.0530880689620972
Epoch 492 / 500 | iteration 5 / 30 | Total Loss: 6.636556625366211 | KNN Loss: 5.590782165527344 | BCE Loss: 1.0457744598388672
Epoch 492 / 500 | iteration 10 / 30 | Total Loss: 6.629528522491455 | KNN Loss: 5.594355583190918 | BCE Loss: 1.035172939300537
Epoch 492 / 500 | iteration 15 / 30 | Total Loss: 6.672906875610352 | KNN Loss: 5.615921974182129 | BCE Loss: 1.0569850206375122
Epoch 492 / 500 | iteration 20 / 30 | Total Loss: 6.664638042449951 | KNN Loss: 5.611959457397461 | BCE Loss: 1.0526784658432007
Epoch 492 / 500 | iteration 25 / 30 | Total Loss: 6.716658115386963 | KNN Loss: 5.670037269592285 | BCE Loss: 1.0466207265853882
Epoch 493 / 500 | iteration 0 / 30 | Total Loss: 6.660101413726807 | KNN Loss: 5.63767147064209 | BC

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 2.5823,  3.8489,  2.5081,  3.2581,  3.1805,  0.6070,  2.5824,  2.1524,
          2.2921,  1.8750,  1.9751,  2.1245,  0.7700,  1.7622,  1.2701,  1.5015,
          2.4763,  2.8642,  2.7307,  2.2322,  1.7439,  2.9127,  2.2424,  2.2796,
          2.2368,  1.7092,  2.0704,  1.2598,  1.4428,  0.3424, -0.2011,  0.9850,
          0.2514,  1.0013,  1.5070,  1.4405,  0.9859,  3.0369,  0.7450,  1.2302,
          0.9363, -0.6310, -0.2804,  2.2822,  2.0094,  0.7214, -0.2034,  0.1042,
          1.4171,  2.4876,  1.8115,  0.1094,  1.3559,  0.4724, -0.6058,  1.1043,
          1.4397,  1.3207,  1.1768,  1.8007,  0.5496,  0.8158,  0.1961,  1.6543,
          1.2789,  1.6702, -1.8123,  0.2765,  1.9163,  2.1375,  2.2309,  0.4083,
          1.2641,  2.4029,  1.9281,  1.2849,  0.2437,  0.7234,  0.2041,  1.5392,
          0.0822,  0.4188,  1.5643, -0.4425,  0.2419, -1.0505, -2.2799, -0.3460,
          0.5775, -1.7547,  0.4210, -0.1554, -0.4991, -0.8217,  0.5404,  1.1915,
         -0.6738, -0.5723,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [12]:
dataset_ = [d[0].cpu() for d in dataset]

In [13]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:01<00:00, 12.72it/s]


In [14]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [15]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [16]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [18]:
tensor_dataset = torch.stack(dataset_)

In [19]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [20]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [21]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [22]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [23]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [24]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [25]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [26]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [27]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [28]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [29]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [30]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [31]:
losses = []
accs = []
sparsity = []

In [32]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
Epoch: 00 | Batch: 000 / 030 | Total loss: 9.652 | Reg loss: 0.009 | Tree loss: 9.652 | Accuracy: 0.000000 | 1.009 sec/iter
Epoch: 00 | Batch: 001 / 030 | Total loss: 9.636 | Reg loss: 0.009 | Tree loss: 9.636 | Accuracy: 0.000000 | 0.728 sec/iter
Epoch: 00 | Batch: 002 / 030 | Total loss: 9.621 | Reg loss: 0.008 | Tree loss: 9.621 | Accuracy: 0.000000 | 0.6 sec/iter
Epoch: 00 | Batch: 003 / 030 | Total loss: 9.605 | Reg loss: 0.008 | Tree loss: 9.605 | Accuracy: 0.000000 | 0.542 sec/iter
Epoch: 00 | Batch: 004 / 030 | Total loss: 9.591 | Reg loss: 0.008 | Tree loss: 9.591 | Accuracy: 0.000000 | 0.509 sec/iter
Epoch: 00 | Batch: 005 / 030 | Total loss: 9.575 | Reg loss: 0.008 | Tree loss: 9.575 | Accuracy: 0.000000 | 0.486 sec/iter
Epoch: 00 | Batch: 006 / 030 | Total loss: 9.561 | Reg loss: 0.008 | Tree loss: 9.561 | Accuracy: 0.000000 | 0.469 sec/iter
Epoch: 00 | Batch: 

Epoch: 02 | Batch: 002 / 030 | Total loss: 9.236 | Reg loss: 0.008 | Tree loss: 9.236 | Accuracy: 0.541016 | 0.359 sec/iter
Epoch: 02 | Batch: 003 / 030 | Total loss: 9.221 | Reg loss: 0.008 | Tree loss: 9.221 | Accuracy: 0.537109 | 0.358 sec/iter
Epoch: 02 | Batch: 004 / 030 | Total loss: 9.206 | Reg loss: 0.008 | Tree loss: 9.206 | Accuracy: 0.587891 | 0.357 sec/iter
Epoch: 02 | Batch: 005 / 030 | Total loss: 9.195 | Reg loss: 0.009 | Tree loss: 9.195 | Accuracy: 0.541016 | 0.356 sec/iter
Epoch: 02 | Batch: 006 / 030 | Total loss: 9.180 | Reg loss: 0.009 | Tree loss: 9.180 | Accuracy: 0.589844 | 0.356 sec/iter
Epoch: 02 | Batch: 007 / 030 | Total loss: 9.160 | Reg loss: 0.009 | Tree loss: 9.160 | Accuracy: 0.568359 | 0.355 sec/iter
Epoch: 02 | Batch: 008 / 030 | Total loss: 9.148 | Reg loss: 0.010 | Tree loss: 9.148 | Accuracy: 0.539062 | 0.356 sec/iter
Epoch: 02 | Batch: 009 / 030 | Total loss: 9.133 | Reg loss: 0.010 | Tree loss: 9.133 | Accuracy: 0.585938 | 0.357 sec/iter
Epoch: 0

Epoch: 04 | Batch: 005 / 030 | Total loss: 8.836 | Reg loss: 0.013 | Tree loss: 8.836 | Accuracy: 0.580078 | 0.347 sec/iter
Epoch: 04 | Batch: 006 / 030 | Total loss: 8.829 | Reg loss: 0.014 | Tree loss: 8.829 | Accuracy: 0.539062 | 0.347 sec/iter
Epoch: 04 | Batch: 007 / 030 | Total loss: 8.806 | Reg loss: 0.014 | Tree loss: 8.806 | Accuracy: 0.570312 | 0.347 sec/iter
Epoch: 04 | Batch: 008 / 030 | Total loss: 8.791 | Reg loss: 0.014 | Tree loss: 8.791 | Accuracy: 0.585938 | 0.347 sec/iter
Epoch: 04 | Batch: 009 / 030 | Total loss: 8.780 | Reg loss: 0.014 | Tree loss: 8.780 | Accuracy: 0.527344 | 0.348 sec/iter
Epoch: 04 | Batch: 010 / 030 | Total loss: 8.756 | Reg loss: 0.015 | Tree loss: 8.756 | Accuracy: 0.562500 | 0.348 sec/iter
Epoch: 04 | Batch: 011 / 030 | Total loss: 8.745 | Reg loss: 0.015 | Tree loss: 8.745 | Accuracy: 0.556641 | 0.348 sec/iter
Epoch: 04 | Batch: 012 / 030 | Total loss: 8.724 | Reg loss: 0.015 | Tree loss: 8.724 | Accuracy: 0.558594 | 0.348 sec/iter
Epoch: 0

Epoch: 06 | Batch: 008 / 030 | Total loss: 8.421 | Reg loss: 0.018 | Tree loss: 8.421 | Accuracy: 0.585938 | 0.351 sec/iter
Epoch: 06 | Batch: 009 / 030 | Total loss: 8.413 | Reg loss: 0.018 | Tree loss: 8.413 | Accuracy: 0.583984 | 0.351 sec/iter
Epoch: 06 | Batch: 010 / 030 | Total loss: 8.383 | Reg loss: 0.019 | Tree loss: 8.383 | Accuracy: 0.613281 | 0.351 sec/iter
Epoch: 06 | Batch: 011 / 030 | Total loss: 8.377 | Reg loss: 0.019 | Tree loss: 8.377 | Accuracy: 0.548828 | 0.351 sec/iter
Epoch: 06 | Batch: 012 / 030 | Total loss: 8.360 | Reg loss: 0.019 | Tree loss: 8.360 | Accuracy: 0.562500 | 0.351 sec/iter
Epoch: 06 | Batch: 013 / 030 | Total loss: 8.336 | Reg loss: 0.020 | Tree loss: 8.336 | Accuracy: 0.562500 | 0.351 sec/iter
Epoch: 06 | Batch: 014 / 030 | Total loss: 8.312 | Reg loss: 0.020 | Tree loss: 8.312 | Accuracy: 0.625000 | 0.351 sec/iter
Epoch: 06 | Batch: 015 / 030 | Total loss: 8.296 | Reg loss: 0.020 | Tree loss: 8.296 | Accuracy: 0.583984 | 0.351 sec/iter
Epoch: 0

Epoch: 08 | Batch: 011 / 030 | Total loss: 7.980 | Reg loss: 0.023 | Tree loss: 7.980 | Accuracy: 0.591797 | 0.353 sec/iter
Epoch: 08 | Batch: 012 / 030 | Total loss: 7.944 | Reg loss: 0.023 | Tree loss: 7.944 | Accuracy: 0.566406 | 0.354 sec/iter
Epoch: 08 | Batch: 013 / 030 | Total loss: 7.936 | Reg loss: 0.023 | Tree loss: 7.936 | Accuracy: 0.572266 | 0.354 sec/iter
Epoch: 08 | Batch: 014 / 030 | Total loss: 7.921 | Reg loss: 0.024 | Tree loss: 7.921 | Accuracy: 0.576172 | 0.354 sec/iter
Epoch: 08 | Batch: 015 / 030 | Total loss: 7.892 | Reg loss: 0.024 | Tree loss: 7.892 | Accuracy: 0.570312 | 0.353 sec/iter
Epoch: 08 | Batch: 016 / 030 | Total loss: 7.879 | Reg loss: 0.024 | Tree loss: 7.879 | Accuracy: 0.554688 | 0.353 sec/iter
Epoch: 08 | Batch: 017 / 030 | Total loss: 7.854 | Reg loss: 0.025 | Tree loss: 7.854 | Accuracy: 0.564453 | 0.353 sec/iter
Epoch: 08 | Batch: 018 / 030 | Total loss: 7.826 | Reg loss: 0.025 | Tree loss: 7.826 | Accuracy: 0.564453 | 0.353 sec/iter
Epoch: 0

...

Epoch: 90 | Batch: 001 / 030 | Total loss: 1.019 | Reg loss: 0.028 | Tree loss: 1.019 | Accuracy: 0.585938 | 0.36 sec/iter
Epoch: 90 | Batch: 002 / 030 | Total loss: 0.999 | Reg loss: 0.028 | Tree loss: 0.999 | Accuracy: 0.609375 | 0.36 sec/iter
Epoch: 90 | Batch: 003 / 030 | Total loss: 1.019 | Reg loss: 0.028 | Tree loss: 1.019 | Accuracy: 0.566406 | 0.36 sec/iter
Epoch: 90 | Batch: 004 / 030 | Total loss: 0.997 | Reg loss: 0.028 | Tree loss: 0.997 | Accuracy: 0.580078 | 0.36 sec/iter
Epoch: 90 | Batch: 005 / 030 | Total loss: 0.997 | Reg loss: 0.028 | Tree loss: 0.997 | Accuracy: 0.564453 | 0.36 sec/iter
Epoch: 90 | Batch: 006 / 030 | Total loss: 0.991 | Reg loss: 0.028 | Tree loss: 0.991 | Accuracy: 0.556641 | 0.36 sec/iter
Epoch: 90 | Batch: 007 / 030 | Total loss: 0.982 | Reg loss: 0.028 | Tree loss: 0.982 | Accuracy: 0.572266 | 0.36 sec/iter
Epoch: 90 | Batch: 008 / 030 | Total loss: 0.947 | Reg loss: 0.028 | Tree loss: 0.947 | Accuracy: 0.601562 | 0.36 sec/iter
Epoch: 90 | Batc

Epoch: 92 | Batch: 004 / 030 | Total loss: 1.004 | Reg loss: 0.027 | Tree loss: 1.004 | Accuracy: 0.582031 | 0.361 sec/iter
Epoch: 92 | Batch: 005 / 030 | Total loss: 0.987 | Reg loss: 0.027 | Tree loss: 0.987 | Accuracy: 0.570312 | 0.361 sec/iter
Epoch: 92 | Batch: 006 / 030 | Total loss: 0.972 | Reg loss: 0.028 | Tree loss: 0.972 | Accuracy: 0.597656 | 0.361 sec/iter
Epoch: 92 | Batch: 007 / 030 | Total loss: 0.968 | Reg loss: 0.028 | Tree loss: 0.968 | Accuracy: 0.583984 | 0.361 sec/iter
Epoch: 92 | Batch: 008 / 030 | Total loss: 0.941 | Reg loss: 0.028 | Tree loss: 0.941 | Accuracy: 0.613281 | 0.361 sec/iter
Epoch: 92 | Batch: 009 / 030 | Total loss: 0.936 | Reg loss: 0.028 | Tree loss: 0.936 | Accuracy: 0.576172 | 0.361 sec/iter
Epoch: 92 | Batch: 010 / 030 | Total loss: 0.922 | Reg loss: 0.028 | Tree loss: 0.922 | Accuracy: 0.593750 | 0.361 sec/iter
Epoch: 92 | Batch: 011 / 030 | Total loss: 0.926 | Reg loss: 0.028 | Tree loss: 0.926 | Accuracy: 0.595703 | 0.361 sec/iter
Epoch: 9

Epoch: 94 | Batch: 007 / 030 | Total loss: 0.966 | Reg loss: 0.027 | Tree loss: 0.966 | Accuracy: 0.564453 | 0.361 sec/iter
Epoch: 94 | Batch: 008 / 030 | Total loss: 0.953 | Reg loss: 0.027 | Tree loss: 0.953 | Accuracy: 0.587891 | 0.361 sec/iter
Epoch: 94 | Batch: 009 / 030 | Total loss: 0.932 | Reg loss: 0.027 | Tree loss: 0.932 | Accuracy: 0.607422 | 0.361 sec/iter
Epoch: 94 | Batch: 010 / 030 | Total loss: 0.930 | Reg loss: 0.028 | Tree loss: 0.930 | Accuracy: 0.580078 | 0.361 sec/iter
Epoch: 94 | Batch: 011 / 030 | Total loss: 0.909 | Reg loss: 0.028 | Tree loss: 0.909 | Accuracy: 0.619141 | 0.361 sec/iter
Epoch: 94 | Batch: 012 / 030 | Total loss: 0.914 | Reg loss: 0.028 | Tree loss: 0.914 | Accuracy: 0.570312 | 0.361 sec/iter
Epoch: 94 | Batch: 013 / 030 | Total loss: 0.904 | Reg loss: 0.028 | Tree loss: 0.904 | Accuracy: 0.570312 | 0.361 sec/iter
Epoch: 94 | Batch: 014 / 030 | Total loss: 0.899 | Reg loss: 0.028 | Tree loss: 0.899 | Accuracy: 0.564453 | 0.361 sec/iter
Epoch: 9

Epoch: 96 | Batch: 010 / 030 | Total loss: 0.919 | Reg loss: 0.027 | Tree loss: 0.919 | Accuracy: 0.595703 | 0.361 sec/iter
Epoch: 96 | Batch: 011 / 030 | Total loss: 0.922 | Reg loss: 0.027 | Tree loss: 0.922 | Accuracy: 0.564453 | 0.361 sec/iter
Epoch: 96 | Batch: 012 / 030 | Total loss: 0.916 | Reg loss: 0.027 | Tree loss: 0.916 | Accuracy: 0.560547 | 0.361 sec/iter
Epoch: 96 | Batch: 013 / 030 | Total loss: 0.900 | Reg loss: 0.028 | Tree loss: 0.900 | Accuracy: 0.582031 | 0.361 sec/iter
Epoch: 96 | Batch: 014 / 030 | Total loss: 0.897 | Reg loss: 0.028 | Tree loss: 0.897 | Accuracy: 0.574219 | 0.361 sec/iter
Epoch: 96 | Batch: 015 / 030 | Total loss: 0.888 | Reg loss: 0.028 | Tree loss: 0.888 | Accuracy: 0.583984 | 0.361 sec/iter
Epoch: 96 | Batch: 016 / 030 | Total loss: 0.910 | Reg loss: 0.028 | Tree loss: 0.910 | Accuracy: 0.517578 | 0.361 sec/iter
Epoch: 96 | Batch: 017 / 030 | Total loss: 0.855 | Reg loss: 0.028 | Tree loss: 0.855 | Accuracy: 0.636719 | 0.361 sec/iter
Epoch: 9

Epoch: 98 | Batch: 013 / 030 | Total loss: 0.917 | Reg loss: 0.027 | Tree loss: 0.917 | Accuracy: 0.541016 | 0.361 sec/iter
Epoch: 98 | Batch: 014 / 030 | Total loss: 0.913 | Reg loss: 0.027 | Tree loss: 0.913 | Accuracy: 0.546875 | 0.361 sec/iter
Epoch: 98 | Batch: 015 / 030 | Total loss: 0.884 | Reg loss: 0.028 | Tree loss: 0.884 | Accuracy: 0.583984 | 0.361 sec/iter
Epoch: 98 | Batch: 016 / 030 | Total loss: 0.874 | Reg loss: 0.028 | Tree loss: 0.874 | Accuracy: 0.587891 | 0.361 sec/iter
Epoch: 98 | Batch: 017 / 030 | Total loss: 0.890 | Reg loss: 0.028 | Tree loss: 0.890 | Accuracy: 0.544922 | 0.361 sec/iter
Epoch: 98 | Batch: 018 / 030 | Total loss: 0.880 | Reg loss: 0.028 | Tree loss: 0.880 | Accuracy: 0.546875 | 0.361 sec/iter
Epoch: 98 | Batch: 019 / 030 | Total loss: 0.849 | Reg loss: 0.028 | Tree loss: 0.849 | Accuracy: 0.585938 | 0.361 sec/iter
Epoch: 98 | Batch: 020 / 030 | Total loss: 0.867 | Reg loss: 0.028 | Tree loss: 0.867 | Accuracy: 0.562500 | 0.361 sec/iter
Epoch: 9

In [33]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [35]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 7.083333333333333


# Extract Rules

# Accumulate samples in the leaves

In [36]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 96


In [37]:
method = 'greedy'

In [38]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/miniconda3/envs/rambo/lib/python3.8/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [40]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== Pattern 25